# 🚖 Advanced NYC Taxi Duration Prediction: Production-Ready ML

## Welcome to the Next Level!

This notebook builds upon our previous regression project for NYC Taxi Trip Duration prediction. Here, we move from **basic model experiments** to **production-grade workflows**. You'll learn how to:

1. Build **scikit-learn pipelines** to streamline preprocessing and modeling.
2. Integrate **MLflow** for experiment tracking and reproducibility.
3. Train **advanced regression models** beyond Linear, Ridge, and Lasso.
4. Perform **cross-validation** and **hyperparameter tuning** for optimal performance.
5. Conduct **robust evaluation and comparison** using multiple metrics.
6. Prepare for deployment with **model versioning and packaging**.

---

## 🎯 Learning Goals

* Understand **modern ML workflows** used in industry.
* Build **scalable and maintainable pipelines** for real-world data.
* Apply **best practices** for model evaluation and reproducibility.
* Transition your ML experiments from **notebook prototypes** to **production-ready systems**.

> ⚡ **Pro Tip:** Keep your code modular and well-documented—this is key to production readiness!

# 🛠️ Production Environment Setup

Before we dive into building **advanced pipelines, models, and tracking experiments**, we need to establish a **robust production-ready environment**. This ensures that:

1. **Experiments are reproducible**  
   - Fixed random seeds, consistent preprocessing, and standardized metrics ensure that results can be trusted and reproduced later.

2. **Our workflow is organized and maintainable**  
   - Proper imports, library versions, and modular code make it easy for teams to collaborate.

3. **Models can be tracked and versioned**  
   - Using **MLflow**, we log experiments, hyperparameters, and metrics for easy comparison and auditability.

4. **Scalability and flexibility**  
   - Preconfigured pipelines, preprocessing tools, and ML libraries allow us to handle larger datasets or swap models seamlessly.

---

> 💡 **Pro Tip:** Think of this step as setting the foundation of a building — if it's strong and well-organized, everything built on top will be reliable, scalable, and easier to maintain.

In [2]:
# 🛠️ Advanced imports for production ML

# Suppress warnings for cleaner outputs
import warnings
warnings.filterwarnings('ignore')

# 🔧 Core Python libraries
import numpy as np           # Efficient numerical computations
import pandas as pd          # Data manipulation and analysis
import matplotlib.pyplot as plt  # Basic plotting
import seaborn as sns        # Advanced visualization
from scipy import stats      # Statistical functions
import joblib               # Save/load large models and preprocessing objects
import json                 # Handle JSON configs and outputs
from datetime import datetime  # Timestamping for logs
import os                   # File system operations
import time                 # Time tracking for experiments

# 🧰 Sklearn libraries - expanded for advanced ML workflows
from sklearn.model_selection import (
    train_test_split,     # Split data into train/test sets
    cross_val_score,      # Cross-validation scoring
    GridSearchCV,         # Hyperparameter tuning (grid search)
    RandomizedSearchCV    # Hyperparameter tuning (randomized search)
)
from sklearn.preprocessing import (
    StandardScaler,       # Feature scaling (zero-mean, unit variance)
    RobustScaler,         # Scaling robust to outliers
    PolynomialFeatures    # Generate polynomial features for non-linear relationships
)
from sklearn.pipeline import Pipeline, FeatureUnion  # Build modular pipelines
from sklearn.compose import ColumnTransformer         # Apply different preprocessing to columns
from sklearn.feature_selection import (
    SelectKBest,          # Univariate feature selection
    f_regression,         # Scoring function for regression
    RFE                   # Recursive feature elimination
)
from sklearn.linear_model import (
    LinearRegression,     # Baseline regression
    Ridge,                # L2-regularized regression
    Lasso,                # L1-regularized regression
    ElasticNet            # Combination of L1 and L2 regularization
)
from sklearn.ensemble import (
    RandomForestRegressor,       # Ensemble of decision trees
    GradientBoostingRegressor,   # Boosted trees for regression
    VotingRegressor              # Combine multiple regressors
)
from sklearn.svm import SVR               # Support Vector Regression
from sklearn.metrics import (
    mean_squared_error,  # Regression metric
    r2_score,            # Regression metric
    mean_absolute_error  # Regression metric
)
from sklearn.inspection import (
    permutation_importance,       # Feature importance
    PartialDependenceDisplay      # Partial dependence plots
)

# 🧪 Advanced model tracking with MLflow
import mlflow                  # Experiment tracking
import mlflow.sklearn          # Log sklearn models
from mlflow.models.signature import infer_signature  # Auto-capture input/output schema for reproducible deployment

# 🎛️ Configuration & Reproducibility

Before we start modeling, it's crucial to **establish reproducible and scalable experiment settings**. This ensures that results are consistent, experiments are traceable, and your workflow is production-ready.

### Key Components:

1. **Reproducibility**
   - `RANDOM_STATE = 42`: Ensures that every run produces the same train/test splits, random sampling, and model results.
   - `TEST_SIZE = 0.2`: Reserves 20% of the data for final evaluation.
   - `VAL_SIZE = 0.2`: Reserves a portion of the training data for validation and hyperparameter tuning.
   - `CV_FOLDS = 5`: Use 5-fold cross-validation to evaluate models robustly.
   - `N_JOBS = -1`: Utilizes all available CPU cores to speed up computations.

2. **Organized Project Structure**
   - `MODEL_DIR = "models"`: All trained models are stored in a dedicated folder.
   - `EXPERIMENT_DIR = "experiments"`: All MLflow experiment logs, metrics, and artifacts are saved in a centralized location.
   - `os.makedirs(..., exist_ok=True)`: Ensures directories exist, preventing file errors during training or logging.

3. **MLflow Experiment Tracking**
   - `mlflow.set_tracking_uri(...)` points MLflow to the experiment folder.
   - `mlflow.set_experiment(experiment_name)` creates a named experiment for this project.
   - This allows you to **log models, metrics, and hyperparameters**, enabling easy comparison across experiments.

> 💡 **Pro Tip:** Proper configuration and tracking is like laying the foundation of a building—everything built on top will be reliable, reproducible, and easy to maintain.

In [3]:
class Config:
    # Reproducibility - Critical for production!
    RANDOM_STATE = 42
    TEST_SIZE = 0.2
    VAL_SIZE = 0.2  # NEW: Validation set for tuning
    CV_FOLDS = 5
    N_JOBS = -1  # Use all available cores
    
    # Model directories - Organized project structure
    MODEL_DIR = "models_nyc_taxi"
    EXPERIMENT_DIR = "experiments_nyc_taxi"
    
    # Create directories if they don't exist
    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(EXPERIMENT_DIR, exist_ok=True)
    
config = Config()

# Initialize MLflow for experiment tracking
mlflow.set_tracking_uri(f"file://{os.path.abspath(config.EXPERIMENT_DIR)}")
experiment_name = "nyc_taxi_duration_advanced"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/1-intro_and_setup/experiments_nyc_taxi/939151119660141953', creation_time=1770039600165, experiment_id='939151119660141953', last_update_time=1770039600165, lifecycle_stage='active', name='nyc_taxi_duration_advanced', tags={}>

# 📊 Loading and Preparing NYC Taxi Dataset

We'll use a sample of the NYC Taxi Trip Duration dataset. This dataset contains information about taxi trips in New York City, with the goal of predicting trip duration.

**Dataset Features:**
- `pickup_longitude`, `pickup_latitude`: Pickup location coordinates
- `dropoff_longitude`, `dropoff_latitude`: Dropoff location coordinates
- `passenger_count`: Number of passengers
- `distance`: Trip distance in miles (calculated from coordinates)
- `pickup_hour`, `pickup_dayofweek`: Temporal features
- `vendor_id`: Taxi vendor identifier

**Target:** `trip_duration` (in minutes)

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elemento/nyc-yellow-taxi-trip-data")

print("Path to dataset files:", path)

Path to dataset files: /home/codespace/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2


In [7]:
import os

path = "/home/codespace/.cache/kagglehub/datasets/elemento/nyc-yellow-taxi-trip-data/versions/2"

sorted(os.listdir(path))


['yellow_tripdata_2015-01.csv',
 'yellow_tripdata_2016-01.csv',
 'yellow_tripdata_2016-02.csv',
 'yellow_tripdata_2016-03.csv']

In [8]:
import pandas as pd

file = f"{path}/yellow_tripdata_2016-01.csv"

chunks = pd.read_csv(
    file,
    chunksize=500_000,
    low_memory=False
)

df1 = next(chunks)
df2 = next(chunks)

df = pd.concat([df1, df2], ignore_index=True)
df.shape


(1000000, 19)

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8


In [10]:
df.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,1.521659,1.738462,3.240361,-72.765110,40.085274,1.048842,-72.818357,40.115370,1.440028,12.785879,0.212128,0.497038,1.531398,0.315954,0.299712,15.642105
std,0.499531,1.329592,3.989969,9.367662,5.160466,0.481473,9.164208,5.048679,0.514079,11.770428,0.264813,0.040916,2.946590,1.944062,0.012749,14.222445
min,1.000000,0.000000,0.000000,-81.101891,0.000000,1.000000,-76.143623,0.000000,1.000000,-300.000000,-0.500000,-0.500000,-3.000000,-17.400000,-0.300000,-300.800000
25%,1.000000,1.000000,1.070000,-73.991386,40.733456,1.000000,-73.990913,40.732273,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,8.000000
50%,2.000000,1.000000,1.820000,-73.981552,40.752167,1.000000,-73.979340,40.752380,1.000000,9.000000,0.000000,0.500000,1.000000,0.000000,0.300000,11.160000
75%,2.000000,2.000000,3.530000,-73.964157,40.768257,1.000000,-73.959373,40.769306,2.000000,14.500000,0.500000,0.500000,2.050000,0.000000,0.300000,17.160000
max,2.000000,8.000000,518.200000,0.000000,57.269276,99.000000,0.000000,48.233334,4.000000,1463.750000,7.000000,0.890000,998.140000,923.580000,0.300000,1463.750000


# 🧠 Advanced Feature Engineering for NYC Taxi Data

For NYC taxi data, we can create meaningful features based on:
1. **Geospatial features**: Distance, direction, borough information
2. **Temporal features**: Time of day, day type, seasonality
3. **Interaction features**: Combining distance with time, passenger effects
4. **Statistical features**: Speed estimates, efficiency metrics

Our custom transformer will create these domain-specific features.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# NEW: Advanced Feature Engineering Class for NYC Taxi
class NYCFeatureEngineer(BaseEstimator, TransformerMixin):
    """Advanced feature engineering for NYC taxi data using domain knowledge"""
    
    def __init__(self):
        self.feature_names = []
    
    def fit(self, X, y=None):
        return self  # No fitting needed for this transformer
    
    def transform(self, X):
        """Transform input data with engineered features"""
        X_df = pd.DataFrame(X, columns=feature_names)
        
        # DOMAIN-DRIVEN FEATURE ENGINEERING:
        
        # 1. Haversine distance (more accurate than Euclidean)
        def haversine_distance(lat1, lon1, lat2, lon2):
            """Calculate haversine distance between two points"""
            R = 3958.8  # Earth radius in miles
            lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
            c = 2 * np.arcsin(np.sqrt(a))
            return R * c
        
        haversine_dist = haversine_distance(
            X_df['pickup_latitude'], X_df['pickup_longitude'],
            X_df['dropoff_latitude'], X_df['dropoff_longitude']
        )
        
        # 2. Direction features (angle)
        delta_lat = X_df['dropoff_latitude'] - X_df['pickup_latitude']
        delta_lon = X_df['dropoff_longitude'] - X_df['pickup_longitude']
        direction_angle = np.arctan2(delta_lat, delta_lon)
        
        # 3. Manhattan distance approximation (NYC streets are grid-like)
        manhattan_dist = (np.abs(delta_lat) * 69 + np.abs(delta_lon) * 53)  # Approx conversion to miles
        
        # 4. Temporal features
        is_rush_hour = ((X_df['pickup_hour'] >= 7) & (X_df['pickup_hour'] <= 9)) | \
                       ((X_df['pickup_hour'] >= 16) & (X_df['pickup_hour'] <= 18))
        is_night = (X_df['pickup_hour'] >= 22) | (X_df['pickup_hour'] <= 5)
        is_weekend = X_df['pickup_dayofweek'].isin([5, 6])
        
        # 5. Centrality features (distance from NYC center - Times Square)
        times_square_lat, times_square_lon = 40.7580, -73.9855
        pickup_from_center = haversine_distance(
            X_df['pickup_latitude'], X_df['pickup_longitude'],
            times_square_lat, times_square_lon
        )
        dropoff_from_center = haversine_distance(
            X_df['dropoff_latitude'], X_df['dropoff_longitude'],
            times_square_lat, times_square_lon
        )
        
        # 6. Efficiency ratio (straight line vs actual)
        efficiency_ratio = haversine_dist / (X_df['distance'] + 1e-8)
        
        # 7. Passenger efficiency (distance per passenger)
        distance_per_passenger = X_df['distance'] / (X_df['passenger_count'] + 1e-8)
        
        # 8. Hour as cyclical feature
        hour_sin = np.sin(2 * np.pi * X_df['pickup_hour'] / 24)
        hour_cos = np.cos(2 * np.pi * X_df['pickup_hour'] / 24)
        
        # Combine all features
        X_eng = np.column_stack([
            X,  # Original features
            haversine_dist.values.reshape(-1, 1),
            direction_angle.values.reshape(-1, 1),
            manhattan_dist.values.reshape(-1, 1),
            is_rush_hour.values.reshape(-1, 1).astype(float),
            is_night.values.reshape(-1, 1).astype(float),
            is_weekend.values.reshape(-1, 1).astype(float),
            pickup_from_center.values.reshape(-1, 1),
            dropoff_from_center.values.reshape(-1, 1),
            efficiency_ratio.values.reshape(-1, 1),
            distance_per_passenger.values.reshape(-1, 1),
            hour_sin.values.reshape(-1, 1),
            hour_cos.values.reshape(-1, 1)
        ])
        
        # Update feature names for interpretability
        self.feature_names = feature_names + [
            'haversine_distance', 'direction_angle', 'manhattan_distance',
            'is_rush_hour', 'is_night', 'is_weekend',
            'pickup_from_center', 'dropoff_from_center',
            'efficiency_ratio', 'distance_per_passenger',
            'hour_sin', 'hour_cos'
        ]
        
        return X_eng

    def fit_transform(self, X, y=None):
        """Mimics sklearn's fit_transform"""
        self.fit(X, y)
        return self.transform(X)
    
    def get_feature_names(self):
        return self.feature_names
        
# Test our feature engineering
engineer = NYCFeatureEngineer()
X_engineered = engineer.fit_transform(X)

print(f"\n🎯 FEATURE ENGINEERING COMPLETE!")
print(f"• Original features: {X.shape[1]}")
print(f"• Engineered features: {X_engineered.shape[1]} (NEW!)")
print(f"• New features created: {engineer.feature_names[-12:]}")

# 🧭 Outlier Handling with IQR — Making the Data More Robust

Taxi data often contains outliers due to GPS errors, incorrect entries, or rare but valid long trips. We'll use IQR-based outlier handling to make our models more robust.

In [ ]:
# NEW: Outlier Handler for Robust Models
class OutlierHandler(BaseEstimator, TransformerMixin):
    """Handle outliers using IQR method - More robust than simple scaling"""
    
    def __init__(self, factor=1.5):
        self.factor = factor
        self.lower_bounds_ = None
        self.upper_bounds_ = None
    
    def fit(self, X, y=None):
        self.lower_bounds_ = []
        self.upper_bounds_ = []
        
        # Calculate IQR bounds for each feature
        for i in range(X.shape[1]):
            Q1 = np.percentile(X[:, i], 25)  # 25th percentile
            Q3 = np.percentile(X[:, i], 75)  # 75th percentile  
            IQR = Q3 - Q1  # Interquartile Range
            self.lower_bounds_.append(Q1 - self.factor * IQR)
            self.upper_bounds_.append(Q3 + self.factor * IQR)
        
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        # Clip values to IQR bounds
        for i in range(X.shape[1]):
            lower = self.lower_bounds_[i]
            upper = self.upper_bounds_[i]
            X_transformed[:, i] = np.clip(X_transformed[:, i], lower, upper)
        
        return X_transformed

# 🏗️ Building the Final Preprocessing Pipeline

We'll combine all steps into a single `Pipeline` object for consistency and reproducibility.

In [ ]:
# Create comprehensive preprocessing pipeline
preprocessor = Pipeline([
    ('feature_engineer', NYCFeatureEngineer()),  # Our new taxi-specific features
    ('outlier_handler', OutlierHandler(factor=1.5)),  # Handle outliers
    ('scaler', RobustScaler())  # Robust to outliers (better than StandardScaler)
])

# Apply preprocessing pipeline
print("🔄 Applying preprocessing pipeline...")
X_processed = preprocessor.fit_transform(X, y)

print("✅ ADVANCED PREPROCESSING PIPELINE BUILT!")
print(f"📊 Processed data shape: {X_processed.shape}")
print(f"🎯 Number of features: {len(preprocessor.named_steps['feature_engineer'].get_feature_names())}")
print("\n📋 All feature names:")
for i, feat in enumerate(preprocessor.named_steps['feature_engineer'].get_feature_names()):
    print(f"  {i+1:2d}. {feat}")

# 🧪 Smarter Data Splitting: Adding a Validation Set

We'll use train/validation/test splits for better model evaluation and hyperparameter tuning.

In [ ]:
# Improved data splitting with validation set
X_temp, X_test, y_temp, y_test = train_test_split(
    X_processed, y, test_size=config.TEST_SIZE, random_state=config.RANDOM_STATE
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=config.VAL_SIZE, random_state=config.RANDOM_STATE
)

print(f"📊 DATA SPLITS (IMPROVED from previous notebook):")
print(f"• Training: {X_train.shape[0]:,} samples (model learning)")
print(f"• Validation: {X_val.shape[0]:,} samples (hyperparameter tuning)") 
print(f"• Test: {X_test.shape[0]:,} samples (final evaluation - NEVER TOUCHED until end)")

# 🧠 Expanding Our Model Arsenal: From Basics to Advanced

We'll use a diverse set of models to find the best performer for taxi duration prediction.

In [ ]:
# define advanced models - Expanded from previous work
advanced_models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(random_state=config.RANDOM_STATE, alpha=1.0),
    'Lasso Regression': Lasso(random_state=config.RANDOM_STATE, alpha=0.1),
    'ElasticNet': ElasticNet(random_state=config.RANDOM_STATE, alpha=0.1, l1_ratio=0.5),  # NEW: Combines L1 + L2
    'Random Forest': RandomForestRegressor(random_state=config.RANDOM_STATE, n_jobs=config.N_JOBS, n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(random_state=config.RANDOM_STATE, n_estimators=100),  # NEW: Sequential learning
    'Support Vector Regression': SVR(kernel='rbf', C=1.0, epsilon=0.1),  # NEW: Different approach
}

# NEW: Voting Ensemble - Combines multiple models
voting_ensemble = VotingRegressor([
    ('ridge', Ridge(random_state=config.RANDOM_STATE, alpha=1.0)),
    ('rf', RandomForestRegressor(random_state=config.RANDOM_STATE, n_jobs=config.N_JOBS, n_estimators=100)),
    ('gb', GradientBoostingRegressor(random_state=config.RANDOM_STATE, n_estimators=100))
])

advanced_models['Voting Ensemble'] = voting_ensemble

print(f"\n🎯 MODEL PORTFOLIO ({len(advanced_models)} models):")
for i, (name, model) in enumerate(advanced_models.items(), 1):
    print(f"  {i:2d}. {name}")

# 🧹 Step-by-Step Model Training and Logging with MLflow

In [ ]:
# ===========================
# 📦 Helper 1 — Basic training and evaluation
# ===========================
def train_and_evaluate(model, X_train, y_train, X_val, y_val):
    """Train model and compute basic metrics on train and validation sets."""
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time

    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    metrics = {
        "train_rmse": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "val_rmse": np.sqrt(mean_squared_error(y_val, y_val_pred)),
        "train_r2": r2_score(y_train, y_train_pred),
        "val_r2": r2_score(y_val, y_val_pred),
        "train_mae": mean_absolute_error(y_train, y_train_pred),
        "val_mae": mean_absolute_error(y_val, y_val_pred),
        "training_time": training_time,
        "overfitting_gap": r2_score(y_train, y_train_pred) - r2_score(y_val, y_val_pred) # EARLY DETECTION FOR OVERFITTING
    }

    return metrics, model

In [ ]:
# ===========================
# 📦 Helper 2 — Cross-validation
# ===========================
def compute_cross_validation(model, X_train, y_train, cv_folds=config.CV_FOLDS):
    """Run cross-validation and return mean and std of R² scores."""
    cv_scores = cross_val_score(model, X_train, y_train,
                                cv=cv_folds, scoring='r2', n_jobs=config.N_JOBS)
    return cv_scores.mean(), cv_scores.std()

In [ ]:
# ===========================
# 📦 Helper 3 — MLflow Logging
# ===========================
def log_to_mlflow(model, metrics, cv_mean, cv_std, run_name):
    """Log params, metrics, and model to MLflow in a clean, minimal way."""
    with mlflow.start_run(run_name=run_name):
        # 1. Log hyperparameters
        mlflow.log_params(model.get_params())
        
        # 2. Log main metrics
        for k, v in metrics.items():
            if k != 'training_time':  # avoid logging long times directly
                mlflow.log_metric(k, float(v))
        mlflow.log_metric("cv_r2_mean", float(cv_mean))
        mlflow.log_metric("cv_r2_std", float(cv_std))
        
        # 3. Save model artifact
        mlflow.sklearn.log_model(model, "model")

In [ ]:
# ===========================
# 📦 Helper 4 — Advanced Model Evaluation
# ===========================
def evaluate_model_advanced(model, X_train, X_val, y_train, y_val, model_name):
    """Train, evaluate, cross-validate, and log model in a clean step-by-step way."""
    # 1. Train and evaluate
    metrics, trained_model = train_and_evaluate(model, X_train, y_train, X_val, y_val)
    
    # 2. Cross-validation
    cv_mean, cv_std = compute_cross_validation(model, X_train, y_train)
    metrics["cv_r2_mean"] = cv_mean
    metrics["cv_r2_std"] = cv_std
    
    # 3. Log everything to MLflow
    log_to_mlflow(model, metrics, cv_mean, cv_std, model_name)
    
    return metrics, trained_model

# 🎯 Run & Track All Advanced Models

In [ ]:
print("🚀 STARTING ADVANCED MODEL EVALUATION...")
results = {}
trained_models = {}

for name, model in advanced_models.items():
    print(f"\n🔧 Training {name}...")
    metrics, trained_model = evaluate_model_advanced(model, X_train, X_val, y_train, y_val, name)
    
    results[name] = metrics
    trained_models[name] = trained_model

    overfit_flag = "⚠️" if metrics['overfitting_gap'] > 0.1 else "✅"
    print(f"✅ {name:20} | Val R²: {metrics['val_r2']:.4f} | "
          f"CV R²: {metrics['cv_r2_mean']:.4f} ± {metrics['cv_r2_std']:.4f} {overfit_flag}")

print("\n📈 All models trained and logged to MLflow!")
print(f"💡 Launch MLflow UI with: mlflow ui --backend-store-uri {config.EXPERIMENT_DIR}")

# 📊 Model Comparison & Visualization

In [ ]:
# ===========================
# 1️⃣ Convert results dict to DataFrame
# ===========================
metrics_df = pd.DataFrame(results).T  # transpose so models are rows
metrics_df = metrics_df[['val_r2', 'val_rmse', 'val_mae', 'overfitting_gap', 'cv_r2_mean']]
metrics_df = metrics_df.sort_values('val_r2', ascending=False)
metrics_df = metrics_df.reset_index().rename(columns={'index': 'Model'})

print("📊 Model comparison table (sorted by Validation R²):")
display(metrics_df)

# ===========================
# 2️⃣ Plot Validation R² (Higher is better)
# ===========================
plt.figure(figsize=(12, 6))
sns.barplot(data=metrics_df, x='Model', y='val_r2', palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title('Validation R² Comparison - NYC Taxi Duration Prediction (Higher is Better ✅)')
plt.ylabel('Validation R²')
plt.xlabel('')
plt.ylim(0, 1)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

# ===========================
# 3️⃣ Plot Validation RMSE (Lower is better)
# ===========================
plt.figure(figsize=(12, 6))
sns.barplot(data=metrics_df, x='Model', y='val_rmse', palette='magma')
plt.xticks(rotation=45, ha='right')
plt.title('Validation RMSE Comparison - NYC Taxi Duration (Lower is Better ✅)')
plt.ylabel('Validation RMSE (minutes)')
plt.xlabel('')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

# ===========================
# 4️⃣ Plot Overfitting Gap (Closer to 0 is better)
# ===========================
plt.figure(figsize=(12, 6))
colors = ['red' if gap > 0.1 else 'green' for gap in metrics_df['overfitting_gap']]
bars = plt.bar(metrics_df['Model'], metrics_df['overfitting_gap'], color=colors, alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.title('Overfitting Gap (train R² - val R²) - NYC Taxi Duration (Closer to 0 is Better ⚖️)')
plt.ylabel('Overfitting Gap')
plt.xlabel('')
plt.axhline(y=0.1, color='red', linestyle='--', alpha=0.8, label='Overfitting Threshold')
plt.axhline(y=-0.1, color='blue', linestyle='--', alpha=0.8, label='Underfitting Threshold')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.show()

# ⚙️ Advanced Hyperparameter Optimization

In [ ]:
# Define comprehensive hyperparameter grids for NYC Taxi data
param_grids = {
    'Random Forest': {
        'n_estimators': [50, 100, 200, 300],  # Number of trees
        'max_depth': [None, 10, 20, 30, 40],       # Tree depth
        'min_samples_split': [2, 5, 10, 20],       # Minimum samples to split
        'min_samples_leaf': [1, 2, 4, 8],         # Minimum samples per leaf
        'max_features': ['auto', 'sqrt', 'log2']  # Features to consider for splits
    },
    
    'Gradient Boosting': {
        'n_estimators': [50, 100, 200, 300],        # Number of boosting stages
        'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],  # Step size shrinkage
        'max_depth': [3, 4, 5, 6, 7],             # Maximum depth per tree
        'min_samples_split': [2, 5, 10, 20],       # Minimum samples to split
        'subsample': [0.8, 0.9, 1.0]           # Fraction of samples for fitting
    },
    
    'Ridge Regression': {
        'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],  # Regularization strength
        'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']  # Algorithm
    },
    
    'Voting Ensemble': {
        'ridge__alpha': [0.1, 1.0, 10.0, 100.0],
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [10, 20, 30],
        'gb__n_estimators': [50, 100, 200],
        'gb__learning_rate': [0.01, 0.05, 0.1, 0.2]
    }
} 

In [ ]:
# Perform hyperparameter optimization
print("🎯 STARTING HYPERPARAMETER OPTIMIZATION...")
tuned_models = {}
optimization_results = {}

for model_name in ['Random Forest', 'Gradient Boosting', 'Ridge Regression', 'Voting Ensemble']:
    print(f"\n🔧 Tuning {model_name}...")
    
    with mlflow.start_run(run_name=f"{model_name}_tuned_nyc"):
        # Use RandomizedSearchCV for efficient optimization
        search = RandomizedSearchCV(
            advanced_models[model_name],
            param_grids[model_name],
            n_iter=30,  # Try 30 random combinations (efficient!)
            cv=config.CV_FOLDS,
            scoring='r2',
            n_jobs=config.N_JOBS,
            random_state=config.RANDOM_STATE,
            verbose=1
        )
        
        # Perform the search
        search.fit(X_train, y_train)
        
        # Store results
        tuned_models[model_name] = search.best_estimator_
        optimization_results[model_name] = {
            'best_score': search.best_score_,
            'best_params': search.best_params_,
            'best_estimator': search.best_estimator_
        }
        
        # Log to MLflow
        mlflow.log_params(search.best_params_)
        mlflow.log_metric('best_cv_score', search.best_score_)
        mlflow.sklearn.log_model(search.best_estimator_, "tuned_model")
        
        print(f"✅ {model_name:20} | Best CV R²: {search.best_score_:.4f}")
        print(f"   Best parameters: {search.best_params_}")

print(f"\n🎉 HYPERPARAMETER OPTIMIZATION COMPLETE!")
print(f"💡 All tuned models saved in MLflow for comparison")

# 📈 Tuned vs Untuned Model Comparison

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error

# ===========================
# 1️⃣ Evaluate tuned models on validation set
# ===========================
tuned_results = {}
for model_name, tuned_model in tuned_models.items():
    y_val_pred = tuned_model.predict(X_val)
    val_r2 = r2_score(y_val, y_val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    improvement = val_r2 - results[model_name]['val_r2']  # vs untuned
    
    tuned_results[model_name] = {
        'val_r2': val_r2,
        'val_rmse': val_rmse,
        'improvement': improvement
    }

# ===========================
# 2️⃣ Print simple comparison table
# ===========================
print(f"{'Model':<20} {'Untuned R²':<12} {'Tuned R²':<12} {'Improvement':<12}")
print("-" * 60)
for model_name, res in tuned_results.items():
    untuned_r2 = results[model_name]['val_r2']
    tuned_r2 = res['val_r2']
    improvement = res['improvement']
    icon = "📈" if improvement > 0.001 else "📉" if improvement < -0.001 else "➡️"
    print(f"{model_name:<20} {untuned_r2:>10.4f} {tuned_r2:>10.4f} {icon} {improvement:>8.4f}")

# ===========================
# 3️⃣ Plot Tuned vs Untuned R²
# ===========================
models = list(tuned_results.keys())
untuned_r2 = [results[m]['val_r2'] for m in models]
tuned_r2 = [tuned_results[m]['val_r2'] for m in models]
improvement = [tuned_results[m]['improvement'] for m in models]

x = np.arange(len(models))
width = 0.35

fig, axes = plt.subplots(1, 2, figsize=(14,5))

# R² comparison
axes[0].bar(x - width/2, untuned_r2, width, label='Untuned', alpha=0.7, color='lightblue')
axes[0].bar(x + width/2, tuned_r2, width, label='Tuned', alpha=0.7, color='lightgreen')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models, rotation=45, ha='right')
axes[0].set_ylabel('Validation R²')
axes[0].set_title('Untuned vs Tuned R² - NYC Taxi Duration (Higher is Better)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Improvement plot
colors = ['green' if i>0.001 else 'red' if i<-0.001 else 'gray' for i in improvement]
axes[1].bar(models, improvement, color=colors, alpha=0.7)
axes[1].axhline(0, color='black', linestyle='--', alpha=0.8)
axes[1].axhline(0.01, color='green', linestyle=':', alpha=0.5, label='Significant Improvement')
axes[1].axhline(-0.01, color='red', linestyle=':', alpha=0.5, label='Significant Degradation')
axes[1].set_ylabel('R² Improvement')
axes[1].set_title('Tuning Impact - NYC Taxi Duration (Green=Better, Red=Worse)')
axes[1].set_xticklabels(models, rotation=45, ha='right')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# ===========================
# 4️⃣ Best tuned model
# ===========================
best_model_name = max(tuned_results, key=lambda m: tuned_results[m]['val_r2'])
best_tuned_model = tuned_models[best_model_name]
print(f"\n🏆 BEST TUNED MODEL: {best_model_name}")
print(f"📊 Validation R²: {tuned_results[best_model_name]['val_r2']:.4f}")
print(f"📈 Improvement over untuned: +{tuned_results[best_model_name]['improvement']:.4f}")
print(f"🔧 Best parameters: {optimization_results[best_model_name]['best_params']}")

# ✅ Final Test Set Evaluation

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Predict on test set
y_test_pred = best_tuned_model.predict(X_test)

# Compute performance metrics
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

print("🔬 FINAL TEST SET EVALUATION")
print("=" * 50)
print(f"📊 Test R²: {test_r2:.4f}")
print(f"📊 Test RMSE: {test_rmse:.4f} minutes")
print(f"📊 Test MAE: {test_mae:.4f} minutes")
print(f"\n📈 For a typical 15-minute taxi ride:")
print(f"   • Average error: ±{test_mae:.1f} minutes ({test_mae/15*100:.1f}% of trip duration)")
print(f"   • RMSE error: ±{test_rmse:.1f} minutes ({test_rmse/15*100:.1f}% of trip duration)")
print(f"\n✅ Model explains {test_r2*100:.1f}% of variance in taxi trip durations")

# 🚀 Model Deployment Preparation

In [ ]:
import os
import json
import joblib
from datetime import datetime

# ===========================
# 0️⃣ Prepare feature names
# ===========================
feature_names_all = preprocessor.named_steps['feature_engineer'].get_feature_names()

# ===========================
# 1️⃣ Identify best tuned model
# ===========================
print(f"🏆 BEST TUNED MODEL: {best_model_name}")
print(f"📊 Validation R²: {tuned_results[best_model_name]['val_r2']:.4f}")
print(f"📊 Test R²: {test_r2:.4f}")

# ===========================
# 2️⃣ Versioning and saving
# ===========================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_version = f"nyc_taxi_v1_{timestamp}"
model_save_dir = os.path.join(config.MODEL_DIR, model_version)
os.makedirs(model_save_dir, exist_ok=True)

# Save model
model_path = os.path.join(model_save_dir, 'best_model.pkl')
joblib.dump(best_tuned_model, model_path)
print(f"✅ Model saved: {model_path}")

# Save preprocessing pipeline
preprocessor_path = os.path.join(model_save_dir, 'preprocessor.pkl')
joblib.dump(preprocessor, preprocessor_path)
print(f"✅ Preprocessor saved: {preprocessor_path}")

# ===========================
# 3️⃣ Create model card
# ===========================
model_card = {
    'model_name': best_model_name,
    'model_version': model_version,
    'timestamp': timestamp,
    'dataset': 'NYC Taxi Trip Duration',
    'target': 'trip_duration (minutes)',
    
    'performance': {
        'test_r2': float(test_r2),
        'test_rmse': float(test_rmse),
        'test_mae': float(test_mae),
        'val_r2': float(tuned_results[best_model_name]['val_r2']),
        'best_cv_score': float(optimization_results[best_model_name]['best_score']),
    },
    
    'data_info': {
        'total_samples': int(len(X)),
        'train_samples': int(len(X_train)),
        'val_samples': int(len(X_val)),
        'test_samples': int(len(X_test)),
        'n_features': X_test.shape[1],
        'original_features': feature_names,
        'engineered_features': feature_names_all[len(feature_names):],
    },
    
    'model_config': {
        'model_class': best_model_name,
        'hyperparameters': dict(best_tuned_model.get_params()) 
            if hasattr(best_tuned_model, 'get_params') else {},
    },
    
    'preprocessing': {
        'steps': [
            'NYCFeatureEngineering (12 new taxi-specific features)',
            'OutlierHandling (IQR method with factor=1.5)',
            'RobustScaler (outlier-resistant scaling)',
        ],
        'engineered_features': [
            'haversine_distance', 'direction_angle', 'manhattan_distance',
            'is_rush_hour', 'is_night', 'is_weekend',
            'pickup_from_center', 'dropoff_from_center',
            'efficiency_ratio', 'distance_per_passenger',
            'hour_sin', 'hour_cos'
        ]
    },
    
    'deployment': {
        'status': 'Ready for Production',
        'recommendations': [
            'Monitor prediction errors in production',
            'Retrain monthly with new taxi data',
            'Alert if RMSE exceeds 8 minutes',
            'Consider time-of-day and traffic patterns for better accuracy'
        ],
        'expected_performance': {
            'r2_range': f'{test_r2*100:.1f}% variance explained',
            'mae_range': f'±{test_mae:.1f} minutes',
            'rmse_range': f'±{test_rmse:.1f} minutes'
        }
    }
}

card_path = os.path.join(model_save_dir, 'model_card.json')
with open(card_path, 'w') as f:
    json.dump(model_card, f, indent=2)
print(f"✅ Model card saved: {card_path}")

# ===========================
# 4️⃣ Deployment requirements
# ===========================
requirements = {
    'python': '3.8+',
    'packages': {
        'scikit-learn': '1.0+',
        'numpy': '1.20+',
        'pandas': '1.3+',
        'joblib': '1.0+',
        'mlflow': '1.0+',
    }
}

req_path = os.path.join(model_save_dir, 'requirements.json')
with open(req_path, 'w') as f:
    json.dump(requirements, f, indent=2)
print(f"✅ Deployment requirements saved: {req_path}")

# ===========================
# 5️⃣ Create example prediction script
# ===========================
example_script = '''# Example: Making predictions with the NYC Taxi Duration model
import joblib
import numpy as np
import pandas as pd

# Load the model and preprocessor
model = joblib.load('best_model.pkl')
preprocessor = joblib.load('preprocessor.pkl')

# Example input (single trip)
example_trip = pd.DataFrame({
    'pickup_longitude': [-73.9855],
    'pickup_latitude': [40.7580],
    'dropoff_longitude': [-73.9772],
    'dropoff_latitude': [40.7527],
    'passenger_count': [2],
    'vendor_id': [1],
    'pickup_hour': [17],  # 5 PM - rush hour
    'pickup_dayofweek': [2],  # Tuesday
    'distance': [2.5]  # miles
})

# Preprocess and predict
X_processed = preprocessor.transform(example_trip.values)
predicted_duration = model.predict(X_processed)

print(f"Predicted trip duration: {predicted_duration[0]:.1f} minutes")
'''

script_path = os.path.join(model_save_dir, 'example_prediction.py')
with open(script_path, 'w') as f:
    f.write(example_script)
print(f"✅ Example prediction script saved: {script_path}")

# ===========================
# 6️⃣ Summary
# ===========================
print("\n" + "=" * 60)
print("💾 DEPLOYMENT PACKAGE READY")
print("=" * 60)
print(f"Location: {model_save_dir}")
print(f"\n📦 Contents:")
print(f"  1. best_model.pkl - Trained {best_model_name}")
print(f"  2. preprocessor.pkl - Complete preprocessing pipeline")
print(f"  3. model_card.json - Model metadata and performance")
print(f"  4. requirements.json - Package dependencies")
print(f"  5. example_prediction.py - Example usage script")
print(f"\n🚀 Ready for deployment in production or Streamlit app!")

# 📚 Key Learnings Recap

This notebook represents a complete production-ready workflow for NYC Taxi Duration prediction. Here's what we accomplished:

1. **✅ Domain-Specific Feature Engineering**  
   - Created 12 taxi-specific features (Haversine distance, rush hour flags, cyclical time features, etc.)
   - Implemented custom transformer for reproducibility

2. **✅ Robust Data Pipeline**  
   - IQR-based outlier handling for taxi data anomalies
   - Robust scaling for better model stability
   - Complete preprocessing pipeline

3. **✅ Comprehensive Model Evaluation**  
   - Compared 8 different regression models
   - Used cross-validation for reliable performance estimates
   - Tracked overfitting with train-val gap analysis

4. **✅ MLflow Experiment Tracking**  
   - Logged all experiments, parameters, and metrics
   - Saved model artifacts for reproducibility
   - Enabled easy comparison of different approaches

5. **✅ Hyperparameter Optimization**  
   - Tuned 4 key models using RandomizedSearchCV
   - Achieved performance improvements through systematic search
   - Identified best model configuration

6. **✅ Production Deployment Preparation**  
   - Model versioning with timestamp
   - Complete model card with metadata
   - Saved pipeline and dependencies
   - Example prediction script

**💡 Summary:**  
This workflow demonstrates how to take real-world taxi data from raw features to a **production-ready prediction system**. The model can now be integrated into applications, dashboards, or APIs for real-time taxi duration predictions.

**Next Steps:**
1. Create a Streamlit app for interactive predictions
2. Deploy as a REST API using FastAPI or Flask
3. Set up monitoring for model performance drift
4. Implement A/B testing for model updates

# 🏋️‍♂️ Exercise: Extend the Workflow

### 🎯 Objective
Extend this workflow with the following enhancements:

1. **Feature Importance Analysis**
   - Use permutation importance or SHAP values
   - Identify which features most influence trip duration

2. **Error Analysis**
   - Analyze where the model makes largest errors
   - Investigate patterns in under/over predictions

3. **Business Metrics**
   - Convert RMSE to business impact (e.g., cost implications)
   - Calculate confidence intervals for predictions

4. **Deployment Script**
   - Create a complete Streamlit app for taxi duration prediction
   - Add visualizations and explanations

### 📝 Instructions
Choose one or more enhancements and implement them in separate notebooks or scripts.

### 📂 Deliverables
1. Extended analysis notebook
2. Streamlit app code
3. Short video demo of the app

> **Goal:** Demonstrate how to take a production ML model and add real business value through analysis, interpretation, and deployment.

In [ ]:
# 🎉 CONGRATULATIONS! You've completed the NYC Taxi Duration Production ML Project!
# Your model is now ready for deployment and real-world use.

print("\n" + "🎉" * 30)
print("  NYC TAXI DURATION PREDICTION PROJECT COMPLETE!")
print("🎉" * 30)

print("\n📊 FINAL RESULTS SUMMARY:")
print(f"   • Best Model: {best_model_name}")
print(f"   • Test R²: {test_r2:.4f} ({test_r2*100:.1f}% variance explained)")
print(f"   • Test RMSE: {test_rmse:.2f} minutes")
print(f"   • Test MAE: {test_mae:.2f} minutes")
print(f"\n💾 Model saved to: {model_save_dir}")
print(f"📈 MLflow experiments: {config.EXPERIMENT_DIR}")
print(f"\n🚀 Ready for production deployment!")